In [5]:
import numpy as np
import matplotlib.pyplot as plt
from database_methods import Table, to_show, head, process_data
import sqlite3
con = sqlite3.connect('database_results.db')
cursor = con.cursor()

In [6]:
mas = Table(connection=con, name='masters')
exp = Table(con, name = 'experiments')
meas = Table(con, name = 'measurements')

In [39]:
print(mas.columns)
print(exp.columns)
print(meas.columns)
print('-'*80)
mas.show_all()
exp.show_all()

CATEGORY, PERIOD, AMPLITUDE, MASTER, FWHM, R, S
DATE, MASTER_ID, DIP, FOLDER
EXPERIMENT_ID, NP, WAVELENGTH, FILE, AR_TH, AR_DIP, AR_NOISE, EXC_THS, RATE, INFO, COUNT, X, Y, CON, INP, INPPX, IBGPX, STD, stdX, stdY, stdCON, stdINP, stdINPPX, stdIBGPX, stdSTD, NOTES, AR_MIN, AR_COND, EXP_TYPE
--------------------------------------------------------------------------------
ID	CATEGORY	PERIOD	AMPLITUDE	MASTER	FWHM	R	S
1	600	405	45	M5	23.0	8.0	15.9
2	600	405	53	P3	25.0	19.7	3.8
3	650	452	32	K4	12.5	11.9	17.8
4	650	452	40	K5	15.0	18.0	8.8
5	750	528	24	L3	7.0	13.8	78.5
6	750	526	30	Q4	9.5	22.0	30.7
ID	DATE	MASTER_ID	DIP	FOLDER
1	20-02-25	2	615	20_02_25_P3/
2	20-02-26	6	727	20_02_26_Q3/
3	20-03-16	4	658	20_03_16_K5/
5	20-02-25	1	598	20_02_25_M5/
6	20-03-16	3	647	20_03_16_K4/
7	20-04-15	5	740	20_04_15_L3/
8	20-04-14	1	606	20_04_14_M5/
9	20-04-20	6	729	20_04_20_Q4/


In [17]:
con.commit()

In [18]:
# 
columns = 'MASTER,  AVG(DIP), AVG(INP), AVG(INP/IBGPX), AVG(CON), AVG(INPPX), AVG(IBGPX)'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
GROUP BY MASTER
ORDER BY DIP DESC
;
   """.format(columns))

head(columns)

for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])


MASTER	 AVG(DI	AVG(INP	AVG(INP	AVG(CON	AVG(INP	AVG(IBG
L3	740.00	18.53	54.94	3.31	1.15	0.3505
Q4	727.80	24.76	68.44	4.60	1.67	0.3943
K5	658.00	14.79	48.99	5.44	1.64	0.3078
K4	647.00	6.18	24.67	3.32	0.9056	0.2691
P3	615.00	5.25	20.50	7.27	1.81	0.2621
M5	606.00	2.54	11.68	6.60	1.42	0.2196


In [52]:
# 
columns = 'MASTER,  FILE,  NP,  X, Y, CON, X*Y, INP, stdINP INPPX, IBGPX, EXP_TYPE'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE MASTER = 'Q4' AND DATE='20-04-20' AND WAVELENGTH = 732 AND Np=80
ORDER BY CON DESC
;
   """.format(columns))

head(columns)

i=0
for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])
    i+=1
    
    
print('---\n{}'.format(i))

MASTER	 FILE	 NP	 X	Y	CON	X*Y	INP	stdINP 	IBGPX	EXP_TYP
Q4	raw_02_	80	24.29	9.28	11.33	225.36	25.61	19.26	0.2904	-
Q4	raw_15_	80	27.30	9.62	11.00	262.61	29.68	22.92	0.2798	sh
Q4	raw_17_	80	25.15	10.14	8.20	254.92	23.66	22.90	0.2759	strep
---
3


In [33]:
meas.commit()

In [25]:
folder = '20_04_20_Q4/'
file = 'raw_17_1'

columns = 'COUNT(MEAS.ID), MEAS.ID'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID 
WHERE FOLDER = '{}' AND FILE = '{}'
;
   """.format(columns, folder, file)
                  )

In [26]:
[d for d in data][0]

(3, 79)

In [20]:
b

In [28]:
columns = 'NP, WAVELENGTH, AR_TH, AR_NOISE, EXC_THS, AR_MIN, AR_COND, exp_type'
data = con.execute("""
SELECT {} FROM 'measurements'
WHERE WAVELENGTH=732
ORDER BY WAVELENGTH ASC;
   """.format(columns))

print(columns.replace(', ','\t'))
for row in data:
    print(str(row).replace(', ', '\t').replace("'", "")[1:-1])

NP	WAVELENGTH	AR_TH	AR_NOISE	EXC_THS	AR_MIN	AR_COND	exp_type
80	732	60.0	0.001	[2]	10	2	-
60	732	59.0	0.001	[2]	10	2	-
60	732	46.0	0.001	[2]	10	2	sh
40	732	12.0	0.001	[1.1]	2	2	-
80	732	100.0	0.001	[2]	2	2	-
60	732	20.0	0.001	[2]	2	2	-
80	732	40.0	0.001	[2]	2	2	strep
147	732	8.0	0.001	[2]	2	2	ps
40	732	9.0	0.001	[2]	6	2	-
80	732	31.0	0.001	[2]	6	2	sh


In [8]:
con.execute("""
UPDATE 'measurements' SET NOTES = 'weak' WHERE COUNT = 165
""")

In [215]:
meas.commit()

In [51]:
con.close()